# Step 7 - Towards Cokriging

In [1]:
# These lines are necessary only if GemPy is not installed
import sys, os
sys.path.append("../../..")
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['MKL_THREADING_LAYER'] = 'GNU'

# Importing GemPy, which takes really long
import gempy as gp

# Importing auxiliary libraries
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from scipy import spatial

In [2]:
domain = pd.read_csv("domain3d.csv")
data = pd.read_csv("data3d.csv")
variogram_fit = pd.read_csv("variogram_fit.csv")
range_ = variogram_fit.iloc[0][0]
sill = variogram_fit.iloc[0][1]
nugget = 5 # have to figure out a real way to do this

In [1]:
def gaussian_variogram_model(d):
    global range_
    global sill
    global nugget
    psill =sill-nugget
    gamma = psill * (1. - np.exp(-d**2./(range_)**2.))+nugget
    return gamma

In [2]:
def exponential_model(d):
    global range_
    global sill
    global nugget
    psill =sill-nugget
    gamma = psill * (1. - np.exp(-d/(range_)))+nugget
    return gamma

In [3]:
def spherical_variogram_model(h):
    global range_
    global sill
    global nugget
    psill =sill-nugget
    if 0 <= abs(h) <= range_:
        gamma = psill * ((3/2*abs(h)/range_)-(1/2*(abs(h)/range_)**3))+nugget
    else:
        gamma = psill + nugget
    return gamma

In [4]:
def create_synthetic_Input_data(n_samples, grid, mean, std):
    '''
    This Part is used to create synthetic Input data. It is possible to vary the number of properties
    and the number of samples. Right now every assigned value is a random value between 0 and 100.
    Data is thus in no way spatially correlated, which makes the Kriging kind of useless, but works for
    testing purposes. Only an isotopic case is realized. Max 10 properties can be used due to a.
    Args: 
        n_prop (int): Number of properties that we have data for
        n_samples (int): Number of sample locations, were we have data for every property (isotopic)
        grid (pandas.dataframe): grid from which the coordinates for the data points are taken
    Returns:
        df_prop(pandas.dataframe): dataframe containing x,y,z coordinates and data for n properties
    '''
    # this case only for one property
    n_prop = 2
    # copy to leave original grid intact
    data_loc_aux = grid.copy()

    # create dataframe with coordinates and n properties
    d = ['X', 'Y', 'Z', 'Property1', 'Property2']
    df_prop = pd.DataFrame(columns=d)
    
    for i in range(n_samples):
            # choose random point from df_sgs_grid_calc
            rand_pos = np.random.randint(0,len(data_loc_aux))
            random_coord = np.array([data_loc_aux.loc[rand_pos,'X'],data_loc_aux.loc[rand_pos,'Y'],data_loc_aux.loc[rand_pos,'Z']])
            
            # delete point from the df sgs_grid_calc
            data_loc_aux = data_loc_aux.drop([rand_pos])
            data_loc_aux = data_loc_aux.reset_index() # reset indicies
            del data_loc_aux['index'] #reset indices
            
            # pick random coordinate from grid
            data_point=([random_coord[0], random_coord[1], random_coord[2]])
            
            if df_prop.empty:
                # create starting point
                data_values = np.random.normal(mean, scale=std)
            else:
                # create a value based on covariance function
                # mean, std = kriging()
                data_value = simple_kriging(data_point, mean, df_prop, std)
            
            # add calculated data value
            data_point.append(data_value)
        
            # put it into the dataframe
            df_prop.loc[i]=data_point

    return df_prop

In [5]:
data = create_synthetic_Input_data(n_samples=200, grid=domain, mean=20, std=5)

NameError: name 'domain' is not defined

In [ ]:
fig = plt.figure(figsize=(16,12))
ax = Axes3D(fig)
ax.axes.set_zlim3d(0,2000)
ax.axes.set_xlim3d(0,3000)
ax.axes.set_ylim3d(0,200)
ax.scatter3D(xs=domain.X,ys=domain.Y,zs=domain.Z, c='red', s=1)
ax.scatter3D(xs=data.X,ys=data.Y,zs=data.Z, c=data.Property, s=50, cmap='prism')